<a href="https://colab.research.google.com/github/PriankaKibria17/Spotify_Analysis/blob/main/Proj2_Pt_4_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This data contains songs that have been played most frequently and have reached the Top 200 Weekly Global charts of spotify.

source: https://www.kaggle.com/datasets/sashankpillai/spotify-top-200-charts-20202021/code 

In [125]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [170]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression 
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.datasets import fetch_openml
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score, classification_report

In [127]:
spotify = pd.read_csv('/content/drive/MyDrive/data/spotify_dataset.csv')
spotify.head()

,Index,Highest Charting Position,Number of Times Charted,Week of Highest Charting,Song Name,Streams,Artist,Artist Followers,Song ID,Genre,...,Danceability,Energy,Loudness,Speechiness,Acousticness,Liveness,Tempo,Duration (ms),Valence,Chord
0,1,1,8,2021-07-23--2021-07-30,Beggin',"48,633,449",Måneskin,3377762,3Wrjm47oTz2sjIgck11l5e,"['indie rock italiano', 'italian pop']",...,0.714,0.8,-4.808,0.0504,0.127,0.359,134.002,211560,0.589,B
1,2,2,3,2021-07-23--2021-07-30,STAY (with Justin Bieber),"47,248,719",The Kid LAROI,2230022,5HCyWlXZPP0y6Gqq8TgA20,['australian hip hop'],...,0.591,0.764,-5.484,0.0483,0.0383,0.103,169.928,141806,0.478,C#/Db
2,3,1,11,2021-06-25--2021-07-02,good 4 u,"40,162,559",Olivia Rodrigo,6266514,4ZtFanR9U6ndgddUvNcjcG,['pop'],...,0.563,0.664,-5.044,0.154,0.335,0.0849,166.928,178147,0.688,A
3,4,3,5,2021-07-02--2021-07-09,Bad Habits,"37,799,456",Ed Sheeran,83293380,6PQ88X9TkUIAUIZJHW2upE,"['pop', 'uk pop']",...,0.808,0.897,-3.712,0.0348,0.0469,0.364,126.026,231041,0.591,B
4,5,5,1,2021-07-23--2021-07-30,INDUSTRY BABY (feat. Jack Harlow),"33,948,454",Lil Nas X,5473565,27NovPIUIRrOZoCHxABJwK,"['lgbtq+ hip hop', 'pop rap']",...,0.736,0.704,-7.409,0.0615,0.0203,0.0501,149.995,212000,0.894,D#/Eb


In [128]:
# There are 3 integer columns and the rest are objects. The data types of some of the numeric columns will have to be changed.
spotify.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1556 entries, 0 to 1555
Data columns (total 23 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Index                      1556 non-null   int64 
 1   Highest Charting Position  1556 non-null   int64 
 2   Number of Times Charted    1556 non-null   int64 
 3   Week of Highest Charting   1556 non-null   object
 4   Song Name                  1556 non-null   object
 5   Streams                    1556 non-null   object
 6   Artist                     1556 non-null   object
 7   Artist Followers           1556 non-null   object
 8   Song ID                    1556 non-null   object
 9   Genre                      1556 non-null   object
 10  Release Date               1556 non-null   object
 11  Weeks Charted              1556 non-null   object
 12  Popularity                 1556 non-null   object
 13  Danceability               1556 non-null   object
 14  Energy  

In [129]:
#No duplicates
spotify.duplicated().sum()

0

In [130]:
#Dropping unneccessary columns with either no distributing values or unique values that are not needed
spotify.drop(columns = ['Index', 'Song ID', 'Chord', 'Week of Highest Charting', 'Weeks Charted', 'Release Date', 'Song Name', 'Artist', 'Genre'], inplace = True)

Changing the data type for:

- streams
- aritst followers
- danceability
- energy
- loudness
- speechiness
- acousticness
- liveness
- tempo
- duration
- valence

Changing the object columns that are numeric.

In [131]:
#removing the the commas in streams
#Code ref: Stackedoverflow
spotify.Streams = spotify.Streams.apply(lambda Streams : Streams.replace(',', ''))

In [132]:
#removing commas, 
spotify.replace(',','', regex=True, inplace=True)
spotify.replace("'",'', regex=True, inplace=True)
spotify.replace(" '' ",'', regex=True, inplace=True)

In [133]:
#renaming column names to remove the space
spotify.rename(columns = {'Highest Charting Position':'HighestChartingPosition'}, inplace = True)
spotify.rename(columns = {'Number of Times Charted':'NumberofTimesCharted'}, inplace = True)
spotify.rename(columns = {'Artist Followers':'ArtistFollowers'}, inplace = True)
spotify.rename(columns = {'Duration (ms)':'Duration(ms)'}, inplace = True)

In [134]:
spotify['Streams'] = spotify['Streams'].astype('int')

In [135]:
#converting datatypes
Allobjectcolumns = ['ArtistFollowers', 'Danceability', 'Energy', 'Loudness', 'Speechiness', 'Acousticness', 'Liveness', 'Tempo', 'Duration(ms)', 'Valence', 'Popularity']
#code ref for loop: Stackedoverflow + Kaggle
for each_col in Allobjectcolumns:
  spotify[each_col] = pd.to_numeric(spotify[each_col], errors = 'coerce')

In [136]:
#identifying null columns
spotify.isnull().sum()

HighestChartingPosition     0
NumberofTimesCharted        0
Streams                     0
ArtistFollowers            11
Popularity                 11
Danceability               11
Energy                     11
Loudness                   11
Speechiness                11
Acousticness               11
Liveness                   11
Tempo                      11
Duration(ms)               11
Valence                    11
dtype: int64

In [137]:
#removing all null values since I the numeric columns are imperative
spotify.dropna(inplace = True)

In [138]:
spotify.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1545 entries, 0 to 1555
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   HighestChartingPosition  1545 non-null   int64  
 1   NumberofTimesCharted     1545 non-null   int64  
 2   Streams                  1545 non-null   int64  
 3   ArtistFollowers          1545 non-null   float64
 4   Popularity               1545 non-null   float64
 5   Danceability             1545 non-null   float64
 6   Energy                   1545 non-null   float64
 7   Loudness                 1545 non-null   float64
 8   Speechiness              1545 non-null   float64
 9   Acousticness             1545 non-null   float64
 10  Liveness                 1545 non-null   float64
 11  Tempo                    1545 non-null   float64
 12  Duration(ms)             1545 non-null   float64
 13  Valence                  1545 non-null   float64
dtypes: float64(11), int64(3)

In [139]:
#the target is streams
X = spotify.drop(columns = 'Streams')
y = spotify['Streams']

In [140]:
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state= 75) 

In [141]:
#getting the shape of the data
print('X Train:', X_train.shape) #the X training set contains 1158 data
print('y Train:', y_train.shape) #the y training set contains 387 data
print('X_test:', X_test.shape)#the X testing set contains 1158 data
print('y_test:', y_test.shape) #the y testing set contains 387 data

X Train: (1158, 13)
y Train: (1158,)
X_test: (387, 13)
y_test: (387,)


In [142]:
#scaling features for future models
scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler()

#modeling

Test 1

In [143]:
rfr = RandomForestRegressor() #instantiating randomforestregressor
rfr_pipe = make_pipeline(scaler, rfr) #creating a pipeline
rfr_pipe.fit(X_train, y_train) #fitting the model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestregressor', RandomForestRegressor())])

In [144]:
#obtaining score for my model
print('Training accuracy:', rfr_pipe.score(X_train, y_train))
print('Testing accuracy:', rfr_pipe.score(X_test, y_test))

Training accuracy: 0.965853733901256
Testing accuracy: 0.8745045406666199


The original score works - but I will attempt to see if I hypertune for a better score

In [161]:
rfr_pipe.get_params() #taking a look at the rfr params to find the ones best suited for optimal results

{'memory': None,
 'randomforestregressor': RandomForestRegressor(),
 'randomforestregressor__bootstrap': True,
 'randomforestregressor__ccp_alpha': 0.0,
 'randomforestregressor__criterion': 'squared_error',
 'randomforestregressor__max_depth': None,
 'randomforestregressor__max_features': 'auto',
 'randomforestregressor__max_leaf_nodes': None,
 'randomforestregressor__max_samples': None,
 'randomforestregressor__min_impurity_decrease': 0.0,
 'randomforestregressor__min_samples_leaf': 1,
 'randomforestregressor__min_samples_split': 2,
 'randomforestregressor__min_weight_fraction_leaf': 0.0,
 'randomforestregressor__n_estimators': 100,
 'randomforestregressor__n_jobs': None,
 'randomforestregressor__oob_score': False,
 'randomforestregressor__random_state': None,
 'randomforestregressor__verbose': 0,
 'randomforestregressor__warm_start': False,
 'standardscaler': StandardScaler(),
 'standardscaler__copy': True,
 'standardscaler__with_mean': True,
 'standardscaler__with_std': True,
 'step

In [164]:
#creatng a list of the parameters I want and naming it params
params = {'randomforestregressor__bootstrap': [True],
          'randomforestregressor__min_samples_leaf': [1],
          'randomforestregressor__min_samples_split': [2],
          'randomforestregressor__n_estimators': [100]}

In [166]:
rfr_grid = GridSearchCV(rfr_pipe, params) #instantiating gridsearch and creating pipeline
rfr_grid.fit(X_train, y_train) #fitting the grid on the model

GridSearchCV(estimator=Pipeline(steps=[('standardscaler', StandardScaler()),
                                       ('randomforestregressor',
                                        RandomForestRegressor())]),
             param_grid={'randomforestregressor__bootstrap': [True],
                         'randomforestregressor__min_samples_leaf': [1],
                         'randomforestregressor__min_samples_split': [2],
                         'randomforestregressor__n_estimators': [100]})

In [167]:
rfr_grid.best_params_ #checking the opitmal combinations of parameters

{'randomforestregressor__bootstrap': True,
 'randomforestregressor__min_samples_leaf': 1,
 'randomforestregressor__min_samples_split': 2,
 'randomforestregressor__n_estimators': 100}

In [168]:
#getting the best versions of the model from gridsearch
best_rfr = rfr_grid.best_estimator_

In [174]:
#fitting the predicitions and getting scores
best_rfr.predict(X_train)
best_rfr.predict(X_test)

print('Training accuracy:', best_rfr.score(X_train, y_train))
print('Testing accuracy:', best_rfr.score(X_test, y_test))

Training accuracy: 0.9684868278552011
Testing accuracy: 0.873736178764275


The score did not have a significant difference, nonetheless still improved.

test 2

In [145]:
linreg = LinearRegression() #instantiating LinearRegression
linregpipe = make_pipeline(scaler, linreg) #creating a pipeline
linregpipe.fit(X_train, y_train) #fitting 

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('linearregression', LinearRegression())])

In [146]:
#obtaining score for my model
print('Training accuracy:', linreg_pipe.score(X_train, y_train))
print('Testing accuracy:', linreg_pipe.score(X_test, y_test))

Training accuracy: 0.1561831274632005
Testing accuracy: 0.17056333091707043


In [175]:
linreg_pipe.get_params()

{'linearregression': LinearRegression(),
 'linearregression__copy_X': True,
 'linearregression__fit_intercept': True,
 'linearregression__n_jobs': None,
 'linearregression__normalize': 'deprecated',
 'linearregression__positive': False,
 'memory': None,
 'standardscaler': StandardScaler(),
 'standardscaler__copy': True,
 'standardscaler__with_mean': True,
 'standardscaler__with_std': True,
 'steps': [('standardscaler', StandardScaler()),
  ('linearregression', LinearRegression())],
 'verbose': False}

In [177]:
#creatng a list of the parameters I want and naming it params again
params = {'linearregression__copy_X': [True],
          'linearregression__fit_intercept': [True],
          'linearregression__normalize': ['deprecated'],
          'linearregression__positive': [False],
          'standardscaler__copy': [True],
          'standardscaler__with_mean': [True],
          'standardscaler__with_std': [True]}

In [178]:
linreg_grid = GridSearchCV(linreg_pipe, params) #instantiating gridsearch and creating pipeline
linreg_grid.fit(X_train, y_train) #fitting the grid on the model

GridSearchCV(estimator=Pipeline(steps=[('standardscaler', StandardScaler()),
                                       ('linearregression',
                                        LinearRegression())]),
             param_grid={'linearregression__copy_X': [True],
                         'linearregression__fit_intercept': [True],
                         'linearregression__normalize': ['deprecated'],
                         'linearregression__positive': [False],
                         'standardscaler__copy': [True],
                         'standardscaler__with_mean': [True],
                         'standardscaler__with_std': [True]})

In [180]:
linreg_grid.best_params_ #checking the opitmal combinations of parameters

{'linearregression__copy_X': True,
 'linearregression__fit_intercept': True,
 'linearregression__normalize': 'deprecated',
 'linearregression__positive': False,
 'standardscaler__copy': True,
 'standardscaler__with_mean': True,
 'standardscaler__with_std': True}

In [181]:
#getting the best versions of the model from gridsearch
best_lr = linreg_grid.best_estimator_

In [182]:
best_lr.predict(X_train)
best_lr.predict(X_test)

print('Training accuracy:', best_lr.score(X_train, y_train))
print('Testing accuracy:', best_lr.score(X_test, y_test))

Training accuracy: 0.15999720440086207
Testing accuracy: 0.16587903261320358


The score remains low, even with the hypertuning.

#Findings

The RandomForestRegression outperformed the LinearRegression model as it received a higher score which means that it predicted the streams well. In attempts to enhance the LinearRegression model the score remains low.